In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,793 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the CSV files
train_csv_file = '/content/drive/MyDrive/Doctor’s Handwritten Prescription BD dataset/Testing/testing_labels.csv'
train_img_file = '/content/drive/MyDrive/Doctor’s Handwritten Prescription BD dataset/Testing/testing_words'
train_output_dir = '/content/drive/MyDrive/edited_trainset'
test_csv_file = '/content/drive/MyDrive/Doctor’s Handwritten Prescription BD dataset/Training/training_labels.csv'
test_img_file = '/content/drive/MyDrive/Doctor’s Handwritten Prescription BD dataset/Training/training_words'
test_output_dir = '/content/drive/MyDrive/edited_testset'

# Load and map labels
data = pd.read_csv(train_csv_file)
train_label_mapping = dict(zip(data['IMAGE'], data['MEDICINE_NAME']))

# Process training images
for image_name in os.listdir(train_img_file):
    image_path = os.path.join(train_img_file, image_name)
    text_label = train_label_mapping.get(image_name, "Unknown")
    image = Image.open(image_path)
    new_image_path = os.path.join(train_output_dir, f"{text_label}.png")
    image.save(new_image_path)

# Load and map labels
data = pd.read_csv(test_csv_file)
test_label_mapping = dict(zip(data['IMAGE'], data['MEDICINE_NAME']))

# Process testing images
for image_name in os.listdir(test_img_file):
    image_path = os.path.join(test_img_file, image_name)
    text_label = test_label_mapping.get(image_name, "Unknown")
    image = Image.open(image_path)
    new_image_path = os.path.join(test_output_dir, f"{text_label}.png")
    image.save(new_image_path)

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory(train_output_dir,
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_output_dir,
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='categorical')

# Define the CNN model
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=len(label_mapping), activation='softmax'))  # Adjust units for multi-class classification

# Compile the model
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

# Example prediction
test_image = tf.keras.preprocessing.image.load_img('/content/drive/MyDrive/Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/1.png', target_size=(64, 64))
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
print(result)
prediction = np.argmax(result, axis=1)

print(prediction)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'label_mapping' is not defined

In [ ]:
!pip install datasets

In [ ]:
from PIL import Image
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset
import os
import shutil
from google.colab import drive

In [ ]:
train_csv_file = '/content/drive/MyDrive/Handwrittenprescription/Testing/testing_labels.csv'
train_img_file = '/content/drive/MyDrive/Handwrittenprescription/Testing/testing_words'
train_output_dir = '/content/drive/MyDrive/edited_trainset'
shutil.rmtree(train_output_dir)
os.makedirs(train_output_dir)

In [ ]:
data = pd.read_csv(train_csv_file)
train_label_mapping = dict(zip(data['IMAGE'], data['MEDICINE_NAME']))

for image_name in os.listdir(train_img_file):
    image_path = os.path.join(train_img_file, image_name)
    text_label = train_label_mapping.get(image_name, "Unknown")
    image = Image.open(image_path)
    new_image_path = os.path.join(train_output_dir, f"{text_label}.png")
    image.save(new_image_path)

In [ ]:
dataset = load_dataset('imagefolder',data_dir = train_output_dir)
if 'validation' not in dataset:
    dataset['validation'] = {}

Resolving data files:   0%|          | 0/79 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    images = [image.convert("RGB") for image in examples['image']]
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    # You need to define how to extract labels from your dataset
    labels = [processor.tokenizer(text, add_special_tokens=True).input_ids for text in train_csv_file['MEDICINE_NAME']]
    return {"pixel_values": pixel_values, "labels": labels}

# Preprocess the dataset
train_dataset = dataset['train'].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)

eval_dataset = dataset['validation'].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['validation'].column_names
)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

TypeError: string indices must be integers

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
)

In [ ]:
trainer.train()

ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [image]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [14]:
image_url = '/content/Sample7.png'
image = Image.open(image_url).convert("RGB")

In [15]:
pixel_values = processor(images=image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [16]:
print(generated_text)

to


## **Model 3**

In [2]:
!pip install datasets
from PIL import Image
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, DatasetDict
import os
import shutil
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
train_csv_file = '/content/drive/MyDrive/Handwrittenprescription/Training/training_labels.csv'
train_img_file = '/content/drive/MyDrive/Handwrittenprescription/Training/training_words'
train_output_dir = '/content/drive/MyDrive/edited_trainset'
test_csv_file = '/content/drive/MyDrive/Handwrittenprescription/Testing/testing_labels.csv'
test_img_file = '/content/drive/MyDrive/Handwrittenprescription/Testing/testing_words'
test_output_dir = '/content/drive/MyDrive/edited_testset'

## **do not run the below code until next text message**

In [3]:
data = pd.read_csv(train_csv_file)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Handwrittenprescription/Training/training_labels.csv'

In [4]:
shutil.rmtree(train_output_dir)
os.makedirs(train_output_dir)
for class_name in data['MEDICINE_NAME'].unique():
    class_dir = os.path.join(train_output_dir, class_name)
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/edited_trainset'

In [ ]:
for index, row in data.iterrows():
    image_name = row['IMAGE']
    class_name = row['MEDICINE_NAME']
    src_path = os.path.join(train_img_file, image_name)
    dest_path = os.path.join(train_output_dir, class_name, image_name)

    if os.path.exists(src_path):
        shutil.move(src_path, dest_path)

In [ ]:
shutil.rmtree(test_output_dir)
os.makedirs(test_output_dir)
data = pd.read_csv(test_csv_file)
for class_name in data['MEDICINE_NAME'].unique():
    class_dir = os.path.join(test_output_dir, class_name)
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)
for index, row in data.iterrows():
    image_name = row['IMAGE']
    class_name = row['MEDICINE_NAME']
    src_path = os.path.join(test_img_file, image_name)
    dest_path = os.path.join(test_output_dir, class_name, image_name)

    if os.path.exists(src_path):
        shutil.move(src_path, dest_path)

## **At any point DO NOT RUN THE ABOVE CODE, it has already been run**

In [ ]:
dataset = load_dataset('imagefolder',data_dir = train_output_dir)
print(dataset)
print(dataset['train'][0])
dataseet = load_dataset('imagefolder',data_dir = test_output_dir)
if 'validation' not in dataset:
    dataset['validation'] = dataseet['train']

Resolving data files:   0%|          | 0/3120 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 3120
    })
})
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=132x81 at 0x7B7A28328AF0>, 'label': 0}


Resolving data files:   0%|          | 0/780 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
def preprocess_function(examples):
    images = [image.convert("RGB") for image in examples['image']]
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    labels = [processor.tokenizer(str(text), add_special_tokens=True).input_ids for text in examples['label']]
    return {"pixel_values": pixel_values, "labels": labels}

train_dataset = dataset['train'].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)

NameError: name 'dataset' is not defined

In [ ]:
eval_dataset = dataset['validation'].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['validation'].column_names
)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

Step,Training Loss


In [21]:
image_url = '/content/Sample10.png'
image = Image.open(image_url).convert("RGB")

In [22]:
pixel_values = processor(images=image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [23]:
final_text = "\n".join(generated_text)
print(generated_text)

1 000


***code from gemini***


In [25]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image # Removed the extra indent from this line

In [27]:
image = Image.open("/content/Sample7.png").convert("RGB")

In [29]:
pixel_values = processor(images=image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [30]:
print(generated_text)

to
